# Datathon Mayo 2018  **_"Haciendo el gasto fiscal visible para el público"_**
## Gasto Neto Mensualizado por año de las Instituciones Públicas

## Lectura de Datos

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import datetime as dt
import pandas_profiling as pd_profiling
import altair as alt

alt.data_transformers.enable('json')
# alt.data_transformers.enable('default', max_rows=10000)

In [2]:
def read_field_type(x):
    '''
    Para facilitar la lectura de los dataframes con los tipos de columna correspondientes.
    '''
    if x in ['String']:
        return str
    elif x in ['Integer', 'Long']:
        return int
    else:
        return str

In [3]:
pub_spend_names = pd.read_csv(os.path.join('data', 
                              'dataset_1_monthly_spend_2009-2017_DICTIONARY.csv'), 
                              sep=';', 
                              encoding='utf-8')

In [4]:
pub_spend_names

,Field name,Field type,Field size,Field format,Range,Description,Example
0,Periodo,Integer,15,#,2009 to 2017,Year of the spend,2009
1,Nombre Partida,String,63,NaN,28 categories,Ministeries,Ministerio de Educación
2,Nombre Capitulo,String,78,NaN,219 categories,Public Services,Fondo Nacional de Salud
3,Nombre Programa,String,78,NaN,363 categories,Programs,Fondo Nacional de Salud
4,Nombre Subtitulo,String,37,NaN,10 categories,First level of the budget classification,TRANSFERENCIAS CORRIENTES
5,Nombre Item,String,54,NaN,43 categories,Second level of the budget classification,Al Sector Privado
6,Nombre Asignación,String,250,NaN,4372 categories,Third level of the budget classification,Subsidios Fondo Solidario de Vivienda
7,Nombre SubAsignación,String,80,NaN,1355 categories,Last level of the budget classification,Asistencia Técnica
8,Mes,String,10,NaN,12 categories,Month,enero
9,Monto,Integer,15,#,(-49702217) to 357607483,"Amount, this data was update by the inflation ...",357607483


In [30]:
pub_spend_names.iloc[9, 5]

"Amount, this data was update by the inflation factors, and it's expressed in thousands of pesos"

In [5]:
pub_spend = pd.read_csv(os.path.join('data', 
                              'dataset_1_monthly_spend_2009-2017.csv'),
                 sep=';', 
                 encoding='utf-8',
                 dtype=dict(zip(pub_spend_names['Field name'], pub_spend_names['Field type'].apply(read_field_type))))

In [6]:
pub_spend.head(5) 

,Periodo,Nombre Partida,Nombre Capitulo,Nombre Programa,Nombre Subtitulo,Nombre Item,Nombre Asignación,Nombre SubAsignación,Mes,Monto_sum
0,2009,Congreso Nacional,Biblioteca Del Congreso,Biblioteca Del Congreso,INICIATIVAS DE INVERSIÓN,Proyectos,Gastos Administrativos,NaN,Abril,0
1,2009,Congreso Nacional,Biblioteca Del Congreso,Biblioteca Del Congreso,INICIATIVAS DE INVERSIÓN,Proyectos,Gastos Administrativos,NaN,Agosto,0
2,2009,Congreso Nacional,Biblioteca Del Congreso,Biblioteca Del Congreso,INICIATIVAS DE INVERSIÓN,Proyectos,Gastos Administrativos,NaN,Diciembre,0
3,2009,Congreso Nacional,Biblioteca Del Congreso,Biblioteca Del Congreso,INICIATIVAS DE INVERSIÓN,Proyectos,Gastos Administrativos,NaN,Enero,0
4,2009,Congreso Nacional,Biblioteca Del Congreso,Biblioteca Del Congreso,INICIATIVAS DE INVERSIÓN,Proyectos,Gastos Administrativos,NaN,Febrero,0


In [7]:
pub_spend.columns = ['periodo', 'partida', 'capitulo', 'programa', 'subtitulo', 'item', 'asignacion', 'subasignacion', 'mes', 'gasto']

In [8]:
pub_spend.shape

(3110448, 10)

In [9]:
pub_spend = pub_spend[pub_spend['gasto'] != 0]

In [10]:
pub_spend.shape

(2107310, 10)

In [11]:
# pub_spend = pub_spend.sample(frac=0.2, random_state=42)
# pub_spend.shape

In [12]:
def mbSize(obj):
    return round(sys.getsizeof(obj) / 1024 / 1024.0, 2)

mbSize(pub_spend)

1425.51

# Estadística descriptiva

## Gasto anual

In [13]:
es_month_dict = {'enero': 1,
                 'febrero': 2,
                 'marzo': 3, 
                 'abril': 4,
                 'mayo': 5,
                 'junio': 6,
                 'julio': 7,
                 'agosto': 8,
                 'septiembre': 9,
                 'octubre': 10,
                 'noviembre': 11,
                 'diciembre': 12
                }

pub_spend['month'] = pub_spend['mes'].str.lower().map(es_month_dict)
pub_spend['day'] = 15
pub_spend['date'] = pd.to_datetime(pub_spend[['day', 'month', 'periodo']].rename(columns={'periodo': 'year'}))
pub_spend.drop(['periodo', 'mes', 'month', 'day'], axis=1, inplace=True)

## Agregaciones

Quitando el cuarto nivel de presupuesto (Subasignacion) se tiene que:

In [16]:
pub_spend_agg3 = pub_spend.groupby(['partida', 'capitulo', 'programa', 'subtitulo', 'item', 'asignacion', 'date'])['gasto'].sum().reset_index()

In [17]:
mbSize(pub_spend_agg3)

909.06

In [18]:
mbSize(pub_spend_agg3) / mbSize(pub_spend) * 100

62.55832197861183

Quitando el cuarto y tercer nivel de presupuesto (Subasignacion y Asignacion) se tiene que:

In [19]:
pub_spend_agg2 = pub_spend.groupby(['partida', 'capitulo', 'programa', 'subtitulo', 'item', 'date'])['gasto'].sum().reset_index()

In [20]:
mbSize(pub_spend_agg2)

261.89

In [21]:
mbSize(pub_spend_agg2) / mbSize(pub_spend) * 100

18.022351597230823

## Gasto Anual

In [27]:
pub_spend_agg2_dict = alt.to_json(pub_spend_agg2, prefix='pub_spend_agg2')

In [ ]:
pub_spend_chart = alt.Chart(pub_spend_agg2_dict['url']).mark_line().encode(
    x=alt.X('date:T', 
        timeUnit='year',
        axis=alt.Axis(title='Year')
           ),
    y='sum(gasto):Q',
).properties( 
    width=600, 
    height=400 
)

In [ ]:
pub_spend_chart

In [34]:
pub_spend_base = alt.Chart().encode(
    x=alt.X('date:T', 
        timeUnit='year',
        axis=alt.Axis(title='Año')
           ),
    y=alt.Y('sum(gasto):Q', axis=alt.Axis(format='.2$', title='Miles de Pesos Chilenos'))
)

chart = alt.layer(
    pub_spend_base.mark_point(),
    pub_spend_base.mark_line(),
    data=pub_spend_agg2_dict['url']
).properties(
    title='Gasto Anual',
    width=600, 
    height=400 
)

In [35]:
chart

## Gasto Año-Mes

In [ ]:
pub_spend_chart.encode(
    x=alt.X('date:T', 
            timeUnit='yearmonth',
            axis=alt.Axis(title='Year-Month')
           )
)

## Gasto por Partida y Año

In [ ]:
pub_spend_chart.mark_bar().encode(
    x=alt.X('sum(gasto):Q'),
    y='partida:N',
    color=alt.Color('date:N', 
            timeUnit='year')
)

## Gasto Anual por Partida

In [ ]:
pub_spend_chart.encode(
    x=alt.X('date:T', 
            timeUnit='year',
            axis=alt.Axis( title='Year')
           ),
    color='partida:N',
)

## Gasto Mensual por Partida en cada Año

In [ ]:
pub_spend_chart.encode(
    x=alt.X('date:T', 
            timeUnit='month',
            axis=alt.Axis( title='Month')
           ),
    color='partida:N',
    row=alt.Row('date:T', 
            timeUnit='year'
           ),
)

## Conteo de registros vs Gasto

### Por Partida

In [ ]:
aux_line = pub_spend_chart.encode(
    x=alt.X('date:T', 
            timeUnit='year',
            axis=alt.Axis( title='Month')
           ),
    y='count()',
    color='partida:N',
)

In [ ]:
aux_line + aux_line.mark_point() | aux_line.encode(y='sum(gasto):Q') + aux_line.mark_point().encode(y='sum(gasto):Q')

### Por capítulo

In [ ]:
# aux_line.encode(color='capitulo') + aux_line.mark_point().encode(color='capitulo')

### Por Subtítulo

In [ ]:
aux_line.encode(color='subtitulo') + aux_line.mark_point().encode(color='subtitulo') | aux_line.encode(y='sum(gasto):Q', color='subtitulo') + aux_line.mark_point().encode(y='sum(gasto):Q', color='subtitulo')

### Por Item

In [ ]:
aux_line.encode(color='item') + aux_line.mark_point().encode(color='item') | aux_line.encode(y='sum(gasto):Q', color='item') + aux_line.mark_point().encode(y='sum(gasto):Q', color='item')

# Perfil por Partida

In [ ]:
partida_name = 'Ministerio De Educación'

In [ ]:
partida = pub_spend[pub_spend['partida'] == partida_name]

In [ ]:
partida_chart = alt.Chart(partida).mark_line().encode(
    x=alt.X('date:T', 
            timeUnit='year',
            axis=alt.Axis( title='Year')
           ),
    y='sum(gasto):Q',
).properties(
    width=600,
    height=400,
)

## Quién gasta

In [ ]:
partida_chart.encode(color='capitulo:N') + partida_chart.mark_point().encode(color='capitulo:N') 

In [ ]:
partida_chart.encode(color='programa:N') + partida_chart.mark_point().encode(color='programa:N') 

## Cómo se gasta

In [ ]:
partida_chart.encode(color='subtitulo:N') + partida_chart.mark_point().encode(color='subtitulo:N')

In [ ]:
partida_chart.encode(color='item:N') + partida_chart.mark_point().encode(color='item:N')